In [9]:
# !pip install opencv-python

In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import cv2
from matplotlib import pyplot as plt
import numpy as np

In [2]:
model = hub.load('https://tfhub.dev/google/movenet/multipose/lightning/1')
movenet = model.signatures['serving_default']

## Code For using camera

In [24]:
cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()

    img = frame.copy()  
    img = tf.image.resize_with_pad(tf.expand_dims(img, axis=0), 192, 256)
    input_img = tf.cast(img,dtype=tf.int32)  
    
    # detection section
    results = movenet(input_img)
    keypoints_with_scores = results['output_0'].numpy()[:,:,:51].reshape((6,17,3))
    
    # render keypoints
    loop_through_people(frame, keypoints_with_scores, EDGES, 0.3)

    cv2.imshow('Movenet MultiPose', frame)
  
    if cv2.waitKey(10) & 0xFF==ord('q'):
        break
cap.release()
cv2.destroyAllWindows()
for i in range (1,5):
    cv2.waitKey(1)


## Code For using video

In [22]:
cap = cv2.VideoCapture('football.mp4')
while cap.isOpened():
    ret, frame = cap.read()

    img = frame.copy()  
    img = tf.image.resize_with_pad(tf.expand_dims(img, axis=0), 352, 640)
    input_img = tf.cast(img,dtype=tf.int32)  
    
    # detection section
    results = movenet(input_img)
    keypoints_with_scores = results['output_0'].numpy()[:,:,:51].reshape((6,17,3))
    
    # render keypoints
    loop_through_people(frame, keypoints_with_scores, EDGES, 0.1)

    cv2.imshow('Movenet MultiPose', frame)
  
    if cv2.waitKey(10) & 0xFF==ord('q'):
        break
cap.release()
cv2.destroyAllWindows()
for i in range (1,5):
    cv2.waitKey(1)


In [21]:
cap.release()
cv2.destroyAllWindows()
for i in range (1,5):
    cv2.waitKey(1)

In [13]:
results['output_0'].numpy()[:,:,:51].reshape((6,17,3))

array([[[5.36000609e-01, 4.95611817e-01, 6.22851014e-01],
        [4.84656870e-01, 5.32136738e-01, 6.09178662e-01],
        [4.84423101e-01, 4.44147408e-01, 7.10799575e-01],
        [4.97068226e-01, 5.80150783e-01, 5.35377800e-01],
        [5.12050092e-01, 3.80608290e-01, 5.10407388e-01],
        [7.07870722e-01, 7.13515759e-01, 8.00658882e-01],
        [7.18279779e-01, 3.25003922e-01, 7.86948919e-01],
        [7.76656508e-01, 8.04676354e-01, 2.51804180e-02],
        [7.70112813e-01, 3.17724675e-01, 5.46771474e-03],
        [7.53964365e-01, 7.19340563e-01, 2.32801065e-02],
        [7.13198423e-01, 4.70181197e-01, 4.64928262e-02],
        [7.95916080e-01, 6.90194130e-01, 4.00912832e-05],
        [7.86388159e-01, 4.63531166e-01, 1.66747759e-05],
        [7.51423359e-01, 6.78015590e-01, 1.40746927e-03],
        [7.46354103e-01, 3.82296264e-01, 3.97907058e-03],
        [3.86938572e-01, 6.01911485e-01, 1.70240796e-03],
        [4.47508961e-01, 5.89648306e-01, 9.53949464e-04]],

       [[3.8

## [nose, left eye, right eye, left ear, right ear, left shoulder, right shoulder, left elbow, right elbow, left wrist, right wrist, left hip, right hip, left knee, right knee, left ankle, right ankle]. The remaining 5 elements [ymin, xmin, ymax, xmax, score] represent the region of the bounding box (in normalized coordinates) and the confidence score of the instance.

In [21]:
keypoints_with_scores.shape

(6, 17, 3)

In [15]:
keypoints_with_scores[0]

array([[0.5293552 , 0.44841567, 0.5881928 ],
       [0.47638726, 0.49198368, 0.7671269 ],
       [0.4674847 , 0.39918908, 0.6593186 ],
       [0.49483964, 0.55325747, 0.6978161 ],
       [0.475124  , 0.3472734 , 0.38256213],
       [0.63364697, 0.65680563, 0.26014534],
       [0.6996311 , 0.20653917, 0.63010436],
       [0.7434801 , 0.9338841 , 0.13660638],
       [0.76694494, 0.14743172, 0.04152149],
       [0.71010214, 0.8250694 , 0.05455555],
       [0.72634023, 0.40200368, 0.03088059],
       [0.8010471 , 0.63170505, 0.00651539],
       [0.7877835 , 0.3853634 , 0.00736779],
       [0.7606194 , 0.69573003, 0.00158437],
       [0.74796444, 0.2949429 , 0.00869546],
       [0.48618102, 0.5794364 , 0.00173463],
       [0.57174945, 0.5704317 , 0.002585  ]], dtype=float32)

In [4]:
# function to loop through each person detected and render
def loop_through_people(frame, keypoints_with_scores, edges, confidence_threshold):
    for person in keypoints_with_scores:
        draw_connections(frame, person, edges, confidence_threshold)
        draw_keypoints(frame, person, confidence_threshold)

# Draw keypoints

In [3]:
def draw_keypoints(frame, keypoints, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for kp in shaped:
        ky, kx, kp_conf = kp
        if kp_conf > confidence_threshold:
            cv2.circle(frame, (int(kx), int(ky)), 4, (0,255,0), -1) 

# Draw Edges

In [5]:
EDGES = {
    (0, 1): 'm',
    (0, 2): 'c',
    (1, 3): 'm',
    (2, 4): 'c',
    (0, 5): 'm',
    (0, 6): 'c',
    (5, 7): 'm',
    (7, 9): 'm',
    (6, 8): 'c',
    (8, 10): 'c',
    (5, 6): 'y',
    (5, 11): 'm',
    (6, 12): 'c',
    (11, 12): 'y',
    (11, 13): 'm',
    (13, 15): 'm',
    (12, 14): 'c',
    (14, 16): 'c'
}

In [6]:
def draw_connections(frame, keypoints, edges, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for edge, color in edges.items():
        p1, p2 = edge
        y1, x1, c1 = shaped[p1]
        y2, x2, c2 = shaped[p2]
        
        if (c1 > confidence_threshold) & (c2 > confidence_threshold):      
            cv2.line(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0,0,255), 2)

In [ ]:
def draw_connections(frame, keypoints, edges, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for edge, color in edges.items():
        p1, p2 = edge
        y1, x1, c1 = shaped[p1]
        y2, x2, c2 = shaped[p2]
        
        if (c1 > confidence_threshold) & (c2 > confidence_threshold):      
            cv2.line(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0,0,255), 2)
# function to loop through each person detected and render
def loop_through_people(frame, keypoints_with_scores, edges, confidence_threshold):
    for person in keypoints_with_scores:
        draw_connections(frame, person, edges, confidence_threshold)
        draw_keypoints(frame, person, confidence_threshold)
def draw_keypoints(frame, keypoints, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for kp in shaped:
        ky, kx, kp_conf = kp
        if kp_conf > confidence_threshold:
            cv2.circle(frame, (int(kx), int(ky)), 4, (0,255,0), -1) 
EDGES = {
(0, 1): 'm',(0, 2): 'c',(1, 3): 'm',(2, 4): 'c',(0, 5): 'm',(0, 6): 'c',(5, 7): 'm',(7, 9): 'm',(6, 8): 'c',(8, 10): 'c',(5, 6): 'y',(5, 11): 'm',(6, 12): 'c',(11, 12): 'y',(11, 13): 'm',(13, 15): 'm',(12, 14): 'c',(14, 16): 'c'
}
cap = cv2.VideoCapture('football.mp4')
while cap.isOpened():
    ret, frame = cap.read()

    img = frame.copy()  
    img = tf.image.resize_with_pad(tf.expand_dims(img, axis=0), 352, 640)
    input_img = tf.cast(img,dtype=tf.int32)  
    
    # detection section
    results = movenet(input_img)
    keypoints_with_scores = results['output_0'].numpy()[:,:,:51].reshape((6,17,3))
    
    # render keypoints
    loop_through_people(frame, keypoints_with_scores, EDGES, 0.1)

    cv2.imshow('Movenet MultiPose', frame)
  
    if cv2.waitKey(10) & 0xFF==ord('q'):
        break
cap.release()
cv2.destroyAllWindows()
for i in range (1,5):
    cv2.waitKey(1)
